In [ ]:
# Testing Cell
from aviary.api import Aircraft
from aviary.subsystems.atmosphere.atmosphere import Atmosphere
from aviary.subsystems.propulsion.propeller.hamilton_standard import (
    HamiltonStandard,
    PostHamiltonStandard,
    PreHamiltonStandard,
)
from aviary.subsystems.propulsion.propeller.propeller_performance import InstallLoss
from aviary.utils.doctape import check_value, get_variable_name, glue_variable

check_value(
    Aircraft.Engine.Propeller.COMPUTE_INSTALLATION_LOSS,
    'aircraft:engine:propeller:compute_installation_loss',
)
glue_variable(get_variable_name(Aircraft.Engine.Propeller.COMPUTE_INSTALLATION_LOSS), md_code=True)
glue_variable(Atmosphere.__name__, md_code=True)
glue_variable(InstallLoss.__name__, md_code=True)
glue_variable(PreHamiltonStandard.__name__, md_code=True)
glue_variable(HamiltonStandard.__name__, md_code=True)
glue_variable(PostHamiltonStandard.__name__, md_code=True)

# Hamilton Standard Propulsion Model

In the 1970s, NASA contracted Hamilton Standard to forecast into the future mid-80s to the 90s what they thought advanced propellers would look like.
The result is what we call “Hamilton Standard model” used in Aviary today.
The [Hamilton Standard Documentation](https://ntrs.nasa.gov/api/citations/19720010354/downloads/19720010354.pdf) is publicly available.
You can find the definitions, methodology, and Fortran code in the document.
In Aviary, we implement only one of the computation options: the code computes the corresponding thrust for a given horsepower.

Below is an XDSM diagram of Hamilton Standard model (assuming {glue:md}`Aircraft.Engine.Propeller.COMPUTE_INSTALLATION_LOSS` is `True`):

![Hamilton Standard Diagram](images/hamilton_standard.png)

The inputs are grouped in three aspects:

Geometric inputs:

- Propeller diameter
- Activity factor per blade (range: 80 to 200, baseline: 150)
- Number of blades (range: 2 to 8)

Power inputs:

- Shaft power to propeller (hp)
- Installation loss factor (0 to 1)

Performance inputs:

- Operating altitude (ft)
- True airspeed (knots)
- Propeller tip speed (Usually < 800 ft/s)
- Integrated lift coefficient (range: 0.3 to 0.8, baseline: 0.5)

Some of the inputs are valid for limited ranges.
When using an odd number of blades, the Hamilton Standard model interpolates using the 2, 4, 6 and 8 blade data.
The corresponding outputs are:

Geometric outputs:

- Design blade pitch angle (at 0.75 Radius)

Power outputs:

- Installation loss factor
- Tip compressibility loss factor
- Power coefficient
- Thrust coefficient (rho=const, no losses)

Performance outputs:

- Flight Mach number
- Propeller tip Mach number
- Advance ratio
- Tip compressibility loss factor
- Thrust
- Propeller efficiency with compressibility losses
- Propeller efficiency with compressibility and installation losses

When shaft power is zero, propeller efficiencies are undefined. We set them as 0.0.

As shown in the above XDSM diagram, the model is an OpenMDAO group that is composed of three components and two groups:

- {glue:md}`Atmosphere` (group)
- {glue:md}`PreHamiltonStandard`
- {glue:md}`HamiltonStandard`
- {glue:md}`InstallLoss` (group)
- {glue:md}`PostHamiltonStandard`

In [ ]:
# Testing Cell
import openmdao.api as om

import aviary.api as av
import aviary.subsystems.propulsion.propeller.hamilton_standard as hs
from aviary.subsystems.atmosphere.atmosphere import Atmosphere
from aviary.subsystems.propulsion.propeller.propeller_performance import (
    InstallLoss,
    PropellerPerformance,
)
from aviary.variable_info.options import get_option_defaults

options = get_option_defaults()
options.set_val(av.Aircraft.Engine.Propeller.COMPUTE_INSTALLATION_LOSS, val=True, units='unitless')
options.set_val(av.Aircraft.Engine.Propeller.NUM_BLADES, val=4, units='unitless')
options.set_val(av.Aircraft.Engine.GENERATE_FLIGHT_IDLE, False)
options.set_val(av.Aircraft.Engine.DATA_FILE, 'models/engines/turboshaft_4465hp.csv')

prob = om.Problem()
group = prob.model
for name in ('traj', 'cruise', 'rhs_all'):
    group = group.add_subsystem(name, om.Group())
var_names = [
    (av.Aircraft.Engine.Propeller.TIP_SPEED_MAX, 0, {'units': 'ft/s'}),
    # (av.Dynamic.Mission.PERCENT_ROTOR_RPM_CORRECTED,0,{'units':'unitless'}),
    (av.Aircraft.Engine.Propeller.ACTIVITY_FACTOR, 0, {'units': 'unitless'}),
    (av.Aircraft.Engine.Propeller.INTEGRATED_LIFT_COEFFICIENT, 0, {'units': 'unitless'}),
]
group.add_subsystem('ivc', om.IndepVarComp(var_names), promotes=['*'])

prob.model.add_subsystem(name='atmosphere', subsys=Atmosphere(num_nodes=1), promotes=['*'])

pp = prob.model.add_subsystem(
    'pp',
    PropellerPerformance(aviary_options=options),
    promotes_inputs=['*'],
    promotes_outputs=['*'],
)
pp.set_input_defaults(av.Aircraft.Engine.Propeller.DIAMETER, 10, units='ft')
pp.set_input_defaults(av.Dynamic.Atmosphere.MACH, 0.7, units='unitless')
# pp.set_input_defaults(av.Dynamic.Atmosphere.TEMPERATURE, 650, units="degR")
pp.set_input_defaults(av.Dynamic.Vehicle.Propulsion.PROPELLER_TIP_SPEED, 800, units='ft/s')
pp.set_input_defaults(av.Dynamic.Mission.VELOCITY, 100, units='knot')
prob.setup()

subsyses = {
    'install_loss': InstallLoss,
    'pre_hamilton_standard': hs.PreHamiltonStandard,
    'hamilton_standard': hs.HamiltonStandard,
    'post_hamilton_standard': hs.PostHamiltonStandard,
}

for name, component in subsyses.items():
    subsys = pp._get_subsystem(name)
    if subsys is None:
        raise ValueError(f"couldn't find {name} in PropellerPerformance")
    if not isinstance(subsys, component):
        raise TypeError(
            f'PropellerPerformance component {name} is {type(subsys)}, but should be {component}'
        )

The {glue:md}`Atmosphere` component provides the flight conditions.
The flight conditions are passed to the {glue:md}`PreHamiltonStandard` component which computes the propeller tip Mach number, advance ratio, and power coefficient.
These values are then fed into the {glue:md}`HamiltonStandard` component.

{glue:md}`HamiltonStandard` is the core of the model.
Given the power coefficient (CP) and advance ratio (J), it finds the blade angle (BL) by a CP-BL chart by tracing the advance ratio.
Then with the blade angle, it finds the thrust coefficient (CT) using its CT-BL chart by tracing advance ratio again.
This algorithm is shown in the below pair of charts.
The CP → BL → CT chart matching algorithm is based on baseline data.
If the user-inputted values are not in the valid region, it will first convert them to those baseline parameters by a sequence of interpolations to do the necessary corrections.
The newly converted parameters are called “effective parameters” (e.g., CPE and CTE).
The outputs are blade angle, thrust coefficient and tip compressibility loss factor.

![CP and CT matching](images/CPE_CTE_matching.png)

Finally, the thrust is computed in the {glue:md}`PostHamiltonStandard` component based on thrust coefficient and tip compressibility loss factor.

The Hamilton Standard model uses wind tunnel test data from uninstalled propellers.
When a nacelle is mounted behind the propeller, an installation loss factor is introduced.
The installation loss factor can be given by the user or computed.
If it is computed, we need another group of components as shown below:

![Installation Loss Factor](images/installation_loss_factor.png)

This diagram is represented by {glue:md}`InstallLoss` group in the first diagram.
Nacelle diameter is needed when installation loss factor is computed.
We use the average nacelle diameter.

The newly added aviary options and variables are:

In [ ]:
# Testing Cell
import inspect

from aviary.api import Aircraft, Dynamic
from aviary.subsystems.propulsion.turboprop_model import TurbopropModel
from aviary.utils.doctape import glue_variable

glue_variable(get_variable_name(TurbopropModel), md_code=True)

# glue all arguments of function TurbopropModel.__init__()
sigs = inspect.signature(TurbopropModel)
parameters = sigs.parameters
for name, param in parameters.items():
    glue_variable(name, md_code=True)
    # print(f'Name: {name}, Default: {param.default}, Kind: {param.kind}')

In [ ]:
Aircraft.Engine.Propeller.DIAMETER
Aircraft.Engine.Propeller.INTEGRATED_LIFT_COEFFICIENT
Aircraft.Engine.Propeller.ACTIVITY_FACTOR
Aircraft.Engine.Propeller.NUM_BLADES
Aircraft.Engine.Propeller.COMPUTE_INSTALLATION_LOSS
Dynamic.Vehicle.Propulsion.PROPELLER_TIP_SPEED
Dynamic.Vehicle.Propulsion.SHAFT_POWER

To build a turboprop engine that uses the Hamilton Standard propeller model we use a {glue:md}`TurbopropModel` object without providing a custom {glue:md}`propeller_model`, here it is set to `None` (the default). In this example, we also set {glue:md}`shaft_power_model` to `None`, another default that assumes we are using a turboshaft engine deck:

In [ ]:
engine = TurbopropModel(options=options, shaft_power_model=None, propeller_model=None)

Some inputs are options:

In [ ]:
options.set_val(Aircraft.Engine.Propeller.DIAMETER, 10, units='ft')
options.set_val(Aircraft.Engine.Propeller.NUM_BLADES, val=4, units='unitless')
options.set_val(Aircraft.Engine.Propeller.COMPUTE_INSTALLATION_LOSS, val=True, units='unitless')

We set the inputs like the following:

In [ ]:
prob.set_val(f'traj.cruise.rhs_all.{Aircraft.Engine.Propeller.TIP_SPEED_MAX}', 710.0, units='ft/s')
prob.set_val(
    f'traj.cruise.rhs_all.{Aircraft.Engine.Propeller.ACTIVITY_FACTOR}', 150.0, units='unitless'
)
prob.set_val(
    f'traj.cruise.rhs_all.{Aircraft.Engine.Propeller.INTEGRATED_LIFT_COEFFICIENT}',
    0.5,
    units='unitless',
)

In [ ]:
# Testing Cell
import aviary.api as av

folder_path = av.get_path('models/engines/propellers')
propellers_dir = folder_path.relative_to(av.top_dir)
glue_variable(propellers_dir, md_code=True)

file_path = av.get_path(folder_path / 'general_aviation.csv')
glue_variable(file_path.name, md_code=True)

map_file_name = file_path.stem + '.map'
file_path = av.get_path(folder_path / map_file_name)
glue_variable(file_path.name, md_code=True)

file_path = av.get_path(folder_path / 'PropFan.csv')
glue_variable(file_path.name, md_code=True)

map_file_name = file_path.stem + '.map'
file_path = av.get_path(folder_path / map_file_name)
glue_variable(file_path.name, md_code=True)

check_value(Aircraft.Engine.Propeller.DATA_FILE, 'aircraft:engine:propeller:data_file')
glue_variable(get_variable_name(Aircraft.Engine.Propeller.DATA_FILE), md_code=True)
check_value(Dynamic.Atmosphere.MACH, 'mach')
glue_variable(get_variable_name(Dynamic.Atmosphere.MACH), md_code=True)

# Propeller Map Alternative

The Hamilton Standard model has limitations where it can be applied; for model aircraft design, it is possible that users may want to provide their own data tables. Two sample data sets are provided in {glue:md}`models/engines/propellers` folder: {glue:md}`general_aviation.csv` and {glue:md}`PropFan.csv`. In both cases, they are in `.csv` format and are converted from `GASP` maps: {glue:md}`general_aviation.csv` and {glue:md}`PropFan.csv` (see [Command Line Tools](aviary_commands.ipynb) for details). The difference between these two samples is that the general aviation sample uses helical Mach numbers as input while the propfan sample uses the free stream Mach numbers. Helical Mach numbers appear higher, due to the inclusion of the rotational component of the tip velocity. In our example, they range from 0.7 to 0.95. To determine which mach type in a GASP map is used, look at the first integer of the first line. If it is 1, it uses helical mach; if it is 2, it uses free stream mach. To determine which mach type is an Aviary propeller file is used, look at which variables are present in the header. It is typically either `helical_mach` or `mach`. If both are present in the header, Aviary will directly use the data in the Mach number column.

To use a propeller map, users can provide the propeller map file path to {glue:md}`Aircraft.Engine.Propeller.DATA_FILE`.

In the Hamilton Standard models, the thrust coefficients do not take compressibility into account. Therefore, propeller tip compressibility loss factor has to be computed and will be used to compute thrust. If a propeller map is used, the compressibility effects should be included in the data provided. Therefore, this factor is assumed to be 1.0 and is supplied to post Hamilton Standard component. Other outputs are computed using the same formulas.